In [ ]:
from topicsfinder import TopicsFinder
from textfilereader import TextFileReader

In [ ]:
reader = TextFileReader('./sample_data/CSS_Hiring_Data_FedEmployee_Reason_OTHER_v1.xlsx')
data = reader.get_dataframe('Reason for filling position(s) with Federal Government Employee -OTHER')
data.head()

In [ ]:
# num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.


In [ ]:
finder = TopicsFinder(data)


In [ ]:
import numpy as np
import pandas as pd
import tqdm
import gensim

grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
# alpha = [0.01]
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
# beta = [0.01]
beta.append('symmetric')
# Validation sets
corpus = finder.corpus
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    # cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                    #                               k=k, a=a, b=b)
                    mod, cv = finder.fit_LDA_model(
                        random_state=100,
                        chunksize=chunksize,
                        passes=passes,
                        iterations=iterations,
                        eval_every=eval_every,
                        num_topics = k,
                        alpha = a,
                        eta = b,

                    )
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv.get_coherence())
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

In [1]:
from topicsfinder import TopicsFinder
from textfilereader import TextFileReader
import optuna

reader = TextFileReader('./sample_data/CSS_Hiring_Data_FedEmployee_Reason_OTHER_v1.xlsx')
data = reader.get_dataframe('Reason for filling position(s) with Federal Government Employee -OTHER')
# reader = TextFileReader('./sample_data/data.xlsx')
# data = reader.get_dataframe('Please briefly describe an example of one burdensome administrative task or process which you believe is "low value"')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kapangyu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kapangyu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kapangyu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def objective(trial):
    k = trial.suggest_int('num_topics', 1, 10)
    a = trial.suggest_discrete_uniform('alpha', 0.01, 1, 0.3)
    b = trial.suggest_discrete_uniform('eta', 0.01, 1, 0.3)
    chunksize = 2000
    passes = 20
    iterations = 400
    eval_every = None  

    finder = TopicsFinder(data)
    _, cv = finder.fit_LDA_model(
        random_state=100,
        chunksize=chunksize,
        passes=passes,
        iterations=iterations,
        eval_every=eval_every,
        num_topics = k,
        alpha = a,
        eta = b,
    )
    return cv.get_coherence()


In [3]:
# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-02-05 13:01:37,701] A new study created in memory with name: no-name-543508f9-06a8-4eae-b18b-183133bf4307
[I 2021-02-05 13:02:17,012] Trial 0 finished with value: 0.20094865437364023 and parameters: {'num_topics': 2, 'alpha': 0.31, 'eta': 0.9099999999999999}. Best is trial 0 with value: 0.20094865437364023.
[I 2021-02-05 13:02:53,290] Trial 1 finished with value: 0.37583864706464165 and parameters: {'num_topics': 10, 'alpha': 0.31, 'eta': 0.61}. Best is trial 1 with value: 0.37583864706464165.
[I 2021-02-05 13:03:22,489] Trial 2 finished with value: 0.3362049638266521 and parameters: {'num_topics': 8, 'alpha': 0.9099999999999999, 'eta': 0.01}. Best is trial 1 with value: 0.37583864706464165.
[I 2021-02-05 13:03:52,108] Trial 3 finished with value: 0.2291558306618219 and parameters: {'num_topics': 2, 'alpha': 0.9099999999999999, 'eta': 0.01}. Best is trial 1 with value: 0.37583864706464165.
[I 2021-02-05 13:04:21,282] Trial 4 finished with value: 0.35707548443040954 and paramete

In [4]:
print(study.best_params)
print(study.best_value)

{'num_topics': 10, 'alpha': 0.01, 'eta': 0.9099999999999999}
0.43024990567547255
